In [1]:
import requests

BASE_URL = "https://services.arcgis.com/acgZYxoN5Oj8pDLa/ArcGIS/rest/services"
OUTPUT = []

def list_services():
    resp = requests.get(BASE_URL, params={"f": "json"})
    resp.raise_for_status()
    return resp.json().get("services", [])

def inspect_service(service):
    name = service["name"]
    service_type = service["type"]
    url = f"{BASE_URL}/{name}/{service_type}"
    try:
        info = requests.get(url, params={"f": "json"}).json()
        capabilities = info.get("capabilities", "")
        layers = info.get("layers", []) or [{"name": info.get("name"), "id": info.get("id", 0)}]
        return {
            "name": name,
            "type": service_type,
            "capabilities": capabilities,
            "num_layers": len(layers),
            "has_attributes": "fields" in info or any("fields" in l for l in layers),
            "queryable": "Query" in capabilities,
            "exportable": "Extract" in capabilities or "ExportTiles" in capabilities
        }
    except Exception as e:
        return {"name": name, "type": service_type, "error": str(e)}

def main():
    services = list_services()
    for service in services:
        info = inspect_service(service)
        OUTPUT.append(info)

    # Print inventory
    for entry in OUTPUT:
        print(f"\n🔹 {entry['name']} ({entry['type']})")
        if "error" in entry:
            print(f"  ⚠️ Error: {entry['error']}")
            continue
        print(f"  Capabilities: {entry['capabilities']}")
        print(f"  Queryable: {entry['queryable']}")
        print(f"  Exportable: {entry['exportable']}")
        print(f"  Layers: {entry['num_layers']}")
        print(f"  Has Attributes: {entry['has_attributes']}")

if __name__ == "__main__":
    main()



🔹 1700_1900_Chloropleth (FeatureServer)
  Capabilities: Query
  Queryable: True
  Exportable: False
  Layers: 1
  Has Attributes: False

🔹 1901_1920_Chloropleth (FeatureServer)
  Capabilities: Query
  Queryable: True
  Exportable: False
  Layers: 1
  Has Attributes: False

🔹 1921_1940_Chloropleth (FeatureServer)
  Capabilities: Query
  Queryable: True
  Exportable: False
  Layers: 1
  Has Attributes: False

🔹 1941_1960_Chloropleth (FeatureServer)
  Capabilities: Query
  Queryable: True
  Exportable: False
  Layers: 1
  Has Attributes: False

🔹 1961_1980_Chloropleth (FeatureServer)
  Capabilities: Query
  Queryable: True
  Exportable: False
  Layers: 1
  Has Attributes: False

🔹 1981_2000_Chloropleth (FeatureServer)
  Capabilities: Query
  Queryable: True
  Exportable: False
  Layers: 1
  Has Attributes: False

🔹 2001_2020_Chloropleth (FeatureServer)
  Capabilities: Query
  Queryable: True
  Exportable: False
  Layers: 1
  Has Attributes: False

🔹 2020_CoCoRaHS_Rainfall_Data (FeatureSe

In [4]:
import requests
import pandas as pd

# Your existing code...
sample_services = [
    "1700_1900_Chloropleth",
    "aiken_gopher_collection_areas",
    "DEV_-_SCDNR_Public_Lands_Viewer",
    "Cemeteries",
    "SC_Coastal_Ponds_2021"
]

BASE_URL = "https://services.arcgis.com/acgZYxoN5Oj8pDLa/ArcGIS/rest/services"

def inspect_layer_fields(service_name):
    layer_url = f"{BASE_URL}/{service_name}/FeatureServer/0"
    try:
        response = requests.get(layer_url, params={"f": "json"})
        response.raise_for_status()
        data = response.json()
        fields = data.get("fields", [])
        has_attributes = any(f.get("name") not in ("OBJECTID", "Shape") for f in fields)
        return {
            "service": service_name,
            "layer_name": data.get("name", "Unknown"),
            "has_attributes": has_attributes,
            "field_names": [f.get("name") for f in fields]
        }
    except Exception as e:
        return {
            "service": service_name,
            "error": str(e)
        }

# Run inspection
results = [inspect_layer_fields(service) for service in sample_services]

# Display results without ace_tools
df = pd.DataFrame(results)
print("Sample FeatureServer Attribute Check")
print("=" * 50)
print(df.to_string(index=False))

# Or if you want a nicer display in Jupyter:
# display(df)

Sample FeatureServer Attribute Check
                        service                    layer_name  has_attributes                                                                                                                                                                                                                                                                                                                                                                                        field_names
          1700_1900_Chloropleth             SC_CountsByCounty            True                                                                                                                                                                                                                                     [OBJECTID, name, county, globalid, Count_1700, Count_1901, Count_1921, Count_1941, Count_1961, Count_1981, Count_2001, Count_2021, Shape__Area, Shape__Length]
  aiken_gopher_collection_areas a

In [5]:
import requests
import pandas as pd
from urllib.parse import urljoin

# Base ArcGIS REST service directory
BASE_URL = "https://services.arcgis.com/acgZYxoN5Oj8pDLa/ArcGIS/rest/services"

# Function to fetch all services
def fetch_services():
    url = f"{BASE_URL}?f=json"
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.json().get("services", [])

# Function to inspect metadata and content
def inspect_service(service):
    name = service["name"]
    type_ = service["type"]
    service_url = f"{BASE_URL}/{name}/{type_}"

    try:
        metadata = requests.get(f"{service_url}?f=json").json()
        description = metadata.get("description") or metadata.get("serviceDescription") or "No description"

        # Try to fetch layer 0
        layer_url = f"{service_url}/0?f=json"
        layer_resp = requests.get(layer_url).json()
        has_data = "fields" in layer_resp and "drawingInfo" in layer_resp
        geometry_type = layer_resp.get("geometryType", "unknown")
        fields = [f['name'] for f in layer_resp.get("fields", [])]

        return {
            "Service Name": name,
            "Type": type_,
            "Has Data": has_data,
            "Description": description,
            "Geometry Type": geometry_type,
            "Field Count": len(fields),
            "Fields": fields,
            "FeatureServer URL": f"{service_url}/0",
            "ArcGIS Viewer": f"https://www.arcgis.com/mapviewer/?layers={service_url}/0"
        }

    except Exception as e:
        return {
            "Service Name": name,
            "Type": type_,
            "Has Data": False,
            "Description": f"Error: {str(e)}",
            "Geometry Type": None,
            "Field Count": 0,
            "Fields": [],
            "FeatureServer URL": f"{service_url}/0",
            "ArcGIS Viewer": "Unavailable"
        }

# Run full inspection
services = fetch_services()
results = [inspect_service(service) for service in services]

# Create DataFrame and display summary
df = pd.DataFrame(results)
df_filtered = df[df["Has Data"] == True].sort_values("Service Name")
df_filtered.reset_index(drop=True, inplace=True)

# Display a summarized view
df_filtered[[
    "Service Name", "Description", "Geometry Type", "Field Count", "FeatureServer URL"
]]


,Service Name,Description,Geometry Type,Field Count,FeatureServer URL
0,1700_1900_Chloropleth,No description,esriGeometryPolygon,14,https://services.arcgis.com/acgZYxoN5Oj8pDLa/A...
1,1901_1920_Chloropleth,No description,esriGeometryPolygon,14,https://services.arcgis.com/acgZYxoN5Oj8pDLa/A...
2,1921_1940_Chloropleth,No description,esriGeometryPolygon,14,https://services.arcgis.com/acgZYxoN5Oj8pDLa/A...
3,1941_1960_Chloropleth,No description,esriGeometryPolygon,14,https://services.arcgis.com/acgZYxoN5Oj8pDLa/A...
4,1961_1980_Chloropleth,No description,esriGeometryPolygon,14,https://services.arcgis.com/acgZYxoN5Oj8pDLa/A...
...,...,...,...,...,...
245,survey123_ee9ded2c3ae345ecaef876c5c90a15be_fie...,View Feature Service (fieldworker) for survey ...,esriGeometryPoint,11,https://services.arcgis.com/acgZYxoN5Oj8pDLa/A...
246,survey123_ef1e16ee554441399666b333003889f6_form,Feature service view of form for the survey ef...,esriGeometryPoint,27,https://services.arcgis.com/acgZYxoN5Oj8pDLa/A...
247,survey123_f0572cfc09154e9893c5a808928378c1_fie...,View Feature Service (fieldworker) for survey ...,esriGeometryPoint,19,https://services.arcgis.com/acgZYxoN5Oj8pDLa/A...
248,taggedSpecies,Testing of tagged marine species at reciever l...,esriGeometryPoint,9,https://services.arcgis.com/acgZYxoN5Oj8pDLa/A...
